## JS: class

## Get class name in constructor

https://github.com/sudheerj/javascript-interview-questions#34-what-is-the-output-of-below-code

In [3]:
class A {
  constructor() {
    console.log(new.target.name);
  }
}

class B extends A {
  constructor() {
    super();
  }
}

new A();
new B();

A
B
B {}


## Mixins

Mixins are not supported in JS but you can write implement multiple inheritance with Proxy:

- https://stackoverflow.com/a/31236132/1760643
- https://github.com/sudheerj/javascript-interview-questions#does-javascript-uses-mixins

In [10]:
function getDesc (obj, prop) {
  var desc = Object.getOwnPropertyDescriptor(obj, prop);
  return desc || (obj=Object.getPrototypeOf(obj) ? getDesc(obj, prop) : void 0);
}
function multiInherit (...protos) {
  return Object.create(new Proxy(Object.create(null), {
    has: (target, prop) => protos.some(obj => prop in obj),
    get (target, prop, receiver) {
      var obj = protos.find(obj => prop in obj);
      return obj ? Reflect.get(obj, prop, receiver) : void 0;
    },
    set (target, prop, value, receiver) {
      var obj = protos.find(obj => prop in obj);
      return Reflect.set(obj || Object.create(null), prop, value, receiver);
    },
    // @ts-ignore
    *enumerate (target) { yield* this.ownKeys(target); },
    ownKeys(target) {
      var hash = Object.create(null);
      for(var obj of protos) for(var p in obj) if(!hash[p]) hash[p] = true;
      return Object.getOwnPropertyNames(hash);
    },
    getOwnPropertyDescriptor(target, prop) {
      var obj = protos.find(obj => prop in obj);
      var desc = obj ? getDesc(obj, prop) : void 0;
      if(desc) desc.configurable = true;
      return desc;
    },
    preventExtensions: (target) => false,
    defineProperty: (target, prop, desc) => false,
  }));
}

// Creating objects
var o1, o2, o3,
    obj = multiInherit(o1={a:1}, o2={b:2}, o3={a:3, b:3});

// Checking property existences
console.log('a' in obj); // true   (inherited from o1)
console.log('b' in obj); // true   (inherited from o2)
console.log('c' in obj); // false  (not found)

// Setting properties
obj.c = 3;

// Reading properties
console.log(obj.a); // 1           (inherited from o1)
console.log(obj.b); // 2           (inherited from o2)
console.log(obj.c); // 3           (own property)
console.log(obj.d); // undefined   (not found)

// The inheritance is "live"
console.log(obj.a); // 1           (inherited from o1)
delete o1.a;
console.log(obj.a); // 3           (inherited from o3)

// Property enumeration
for(var p in obj) p; // "c", "b", "a"

true
true
false
1
2
3
undefined
1
3
